In [18]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn import feature_selection
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn import impute
from sklearn.pipeline import Pipeline
from sklearn import compose
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import median_absolute_error
# from sklearn.metrics import median_absolute_error
# from sklearn.model_selection import cross_val_score

In [19]:
data = pd.read_csv("../data/data_cleaned.csv")

In [20]:
df = data

In [23]:
df

,Age,Nationality,Overall,Potential,Club,Value,Wage,Special,Preferred_Foot,International_Reputation,...,GKReflexes,Release_Clause,League_Country,League_Level_Pyramid,League_Revenue,League_Revenue_Per_Team,Is_Goalkeeper,Loaned_Out,Off_Work_Rate,Def_Work_Rate
0,31,6,94,94,212,110.50,0.565,2202,0,5.0,...,8.0,226.500,Spain,1.0,4479.0,223.95,0,0,2,2
1,33,122,94,94,326,77.00,0.405,2228,1,5.0,...,11.0,127.100,Italy,1.0,2163.0,108.20,0,0,0,1
2,26,20,92,93,435,118.50,0.290,2143,1,5.0,...,11.0,228.100,France,1.0,1692.0,84.60,0,0,0,2
3,27,138,91,93,375,72.00,0.260,1471,1,4.0,...,94.0,138.600,England,1.0,6562.0,328.10,1,0,2,2
4,27,13,91,92,374,102.00,0.355,2281,1,4.0,...,13.0,196.400,England,1.0,6562.0,328.10,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17950,19,46,47,65,165,0.06,0.001,1307,1,1.0,...,9.0,0.143,England,4.0,106.0,4.40,0,0,2,2
17951,19,143,47,63,588,0.06,0.001,1098,1,1.0,...,12.0,0.113,Sweden,1.0,145.0,9.10,0,0,2,2
17952,16,46,47,67,122,0.06,0.001,1189,1,1.0,...,13.0,0.165,England,4.0,106.0,4.40,0,0,2,2
17953,17,46,47,66,587,0.06,0.001,1228,1,1.0,...,9.0,0.143,England,4.0,106.0,4.40,0,0,2,2


In [24]:
# replace nan
cat_col = ['Nationality', 'Club', 'Preferred_Foot', 'Off_Work_Rate','Def_Work_Rate', 
           'Real_Face', 'Position', 'Loaned_Out', 'League_Country']
df[cat_col] = df[cat_col].fillna(value='nan')  # fill na for categorical features
# df = df.fillna(value=df.median())  # fill na for numerical features

In [25]:
# label encoding
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for c in cat_col:
    df[c] = le.fit_transform(df[c])

In [26]:
regressor = RandomForestRegressor(n_estimators=30,
                                  min_samples_leaf=1,
                                  random_state=42)

In [27]:
numerical_col =  list(set(df.columns) - set(cat_col))
numerical_col.remove('Value')

In [28]:
def make_pipeline(regressor=None):
    "Create a single pipeline that processing the data and then fits the regressor." 
    
    median_impute = impute.SimpleImputer(strategy='median')

    numerical_names = numerical_col
    
    col_transform = compose.ColumnTransformer([('numerics',median_impute,numerical_names)])

    steps_list = [('col_t',col_transform),('clf',regressor)]

    pipeline = Pipeline(steps=steps_list)
    
    return pipeline

In [29]:
pipeline = make_pipeline(regressor)

In [30]:
y = df['Value']

In [31]:
X = df.drop('Value',axis=1)

In [32]:
kfold= StratifiedKFold(n_splits=5,shuffle=True)

In [37]:
scores = cross_val_score(pipeline, X, y, cv=5, scoring=make_scorer(median_absolute_error))

In [38]:
np.mean(scores)

0.3867999999999998

In [39]:
scores

array([1.59333333, 0.17666667, 0.08666667, 0.04233333, 0.035     ])